Plot ATLAS Compact
==================

Check if a given point is within a tide model domain

OTIS format tidal solutions provided by Ohio State University and ESR  
- http://volkov.oce.orst.edu/tides/region.html  

#### Python Dependencies
 - [numpy: Scientific Computing Tools For Python](https://www.numpy.org)  
 - [scipy: Scientific Tools for Python](https://www.scipy.org/)  
 - [pyproj: Python interface to PROJ library](https://pypi.org/project/pyproj/)  
 - [netCDF4: Python interface to the netCDF C library](https://unidata.github.io/netcdf4-python/)  
 - [matplotlib: Python 2D plotting library](https://matplotlib.org/)  

#### Program Dependencies

- `convert_crs.py`: convert points to and from Coordinates Reference Systems  
- `io.model.py`: retrieves tide model parameters for named tide models
- `io.OTIS.py`: extract tidal harmonic constants from OTIS tide models  
- `predict.py`: predict tidal values using harmonic constants  
- `time.py`: utilities for calculating time operations

This notebook uses Jupyter widgets to set parameters for calculating the tidal maps.

#### Load modules

In [ ]:
from __future__ import print_function

import os
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

import pyTMD.time
import pyTMD.tools
import pyTMD.io

# autoreload
%load_ext autoreload
%autoreload 2

In [ ]:
# available model list
model_list = sorted(pyTMD.io.model.ATLAS_compact())
# display widgets for setting directory and model
TMDwidgets = pyTMD.tools.widgets()
TMDwidgets.model.options = model_list
TMDwidgets.model.value = 'TPXO8-atlas'
TMDwidgets.compress.value = False
widgets.VBox([
    TMDwidgets.directory,
    TMDwidgets.model,
    TMDwidgets.compress,
])

In [ ]:
# get model parameters
model = pyTMD.io.model(TMDwidgets.directory.value,
    compressed=TMDwidgets.compress.value
   ).elevation(TMDwidgets.model.value)
# read each constituent
constituents,nc = pyTMD.read_tide_model.read_constituents(model.model_file)

In [ ]:
# if reading a global solution with localized solutions
x0,y0,hz0,mz0,iob,dt,pmask,local = pyTMD.io.OTIS.read_atlas_grid(model.grid_file)
xi,yi,hz = pyTMD.io.OTIS.combine_atlas_model(x0,y0,hz0,pmask,local,VARIABLE='depth')
mz = pyTMD.io.OTIS.create_atlas_mask(x0,y0,mz0,local,VARIABLE='depth')
# resample global solution to 2 arc-minute solution
x30,y30,hz30 = pyTMD.io.OTIS.interpolate_atlas_model(x0,y0,hz0)

In [ ]:
%matplotlib widget
# plot the ATLAS mask
fig,ax1 = plt.subplots(num=1, figsize=(8.25,5.25), dpi=120)
ax1.imshow(mz, interpolation='nearest',
    extent=(xi.min(),xi.max(),yi.min(),yi.max()),
    vmin=0, vmax=1, origin='lower', cmap='gray_r', alpha=0.5)
ax1.imshow(pmask, interpolation='nearest',
    extent=(x0.min(),x0.max(),y0.min(),y0.max()),
    vmin=0, vmax=1, origin='lower', cmap='Purples', alpha=0.5)
# no ticks on the x and y axes
ax1.get_xaxis().set_ticks([])
ax1.get_yaxis().set_ticks([])
# stronger linewidth on frame
[i.set_linewidth(2.0) for i in ax1.spines.values()]
# adjust subplot within figure
fig.subplots_adjust(left=0.02,right=0.98,bottom=0.05,top=0.98)
plt.show()

In [ ]:
# percent difference between grids
percent = 100.0*(hz30 - hz)/hz30
# plot the percent difference between ATLAS depth
fig,ax2 = plt.subplots(num=2, figsize=(8.25,5.25), dpi=120)
im = ax2.imshow(percent, interpolation='nearest',
    extent=(xi.min(),xi.max(),yi.min(),yi.max()),
    vmin=-40, vmax=40, origin='lower', cmap='PRGn')
# Add colorbar and adjust size
# pad = distance from main plot axis
# extend = add extension triangles to upper and lower bounds
# options: neither, both, min, max
# shrink = percent size of colorbar
# aspect = lengthXwidth aspect of colorbar
cbar = plt.colorbar(im, ax=ax2, pad=0.025, extend='both',
    extendfrac=0.0375, orientation='horizontal', shrink=0.925,
    aspect=22, drawedges=False)
# rasterized colorbar to remove lines
cbar.solids.set_rasterized(True)
# Add label to the colorbar
cbar.ax.set_xlabel(f'{model.name} Bathymetry Differences', fontsize=13)
cbar.ax.set_ylabel('%', fontsize=13, rotation=0)
cbar.ax.yaxis.set_label_coords(1.0325, 0.15)
# ticks lines all the way across
cbar.ax.tick_params(which='both', width=1, length=23,
    labelsize=13, direction='in')
# no ticks on the x and y axes
ax2.get_xaxis().set_ticks([])
ax2.get_yaxis().set_ticks([])
# stronger linewidth on frame
[i.set_linewidth(2.0) for i in ax2.spines.values()]
# adjust subplot within figure
fig.subplots_adjust(left=0.02,right=0.98,bottom=0.05,top=0.98)
plt.show()

In [ ]:
# total amplitude difference for all constituents
diff = np.zeros_like(hz30)
power = np.zeros_like(hz30)
# for each constituent
for i,c in enumerate(constituents):
    # if reading a global solution with localized solutions
    z0,zlocal = pyTMD.io.OTIS.read_atlas_elevation(model.model_file,i,c)
    xi,yi,z = pyTMD.io.OTIS.combine_atlas_model(x0,y0,z0,pmask,zlocal,VARIABLE='z')
    # resample global solution to 2 arc-minute solution
    x30,y30,z30 = pyTMD.io.OTIS.interpolate_atlas_model(x0,y0,z0)
    # add to total amplitude difference
    diff += (z30.real - z.real)**2 + (z30.imag - z.imag)**2
    power += z30.real**2 + z30.imag**2

In [ ]:
# calculate the percent difference
percent = 100.0*np.sqrt(diff/power)
# plot the percent between ATLAS tidal amplitudes
fig,ax3 = plt.subplots(num=3, figsize=(8.25,5.25), dpi=120)
im = ax3.imshow(percent, interpolation='nearest',
    extent=(xi.min(),xi.max(),yi.min(),yi.max()),
    vmin=0, vmax=100, origin='lower', cmap='BuPu')
# Add colorbar and adjust size
# pad = distance from main plot axis
# extend = add extension triangles to upper and lower bounds
# options: neither, both, min, max
# shrink = percent size of colorbar
# aspect = lengthXwidth aspect of colorbar
cbar = plt.colorbar(im, ax=ax3, pad=0.025, extend='max',
    extendfrac=0.0375, orientation='horizontal', shrink=0.925,
    aspect=22, drawedges=False)
# rasterized colorbar to remove lines
cbar.solids.set_rasterized(True)
# Add label to the colorbar
cbar.ax.set_xlabel(f'{model.name} Tide Height Differences', fontsize=13)
cbar.ax.set_ylabel('%', fontsize=13, rotation=0)
cbar.ax.yaxis.set_label_coords(1.0325, 0.15)
# ticks lines all the way across
cbar.ax.tick_params(which='both', width=1, length=23,
    labelsize=13, direction='in')
# no ticks on the x and y axes
ax3.get_xaxis().set_ticks([])
ax3.get_yaxis().set_ticks([])
# stronger linewidth on frame
[i.set_linewidth(2.0) for i in ax3.spines.values()]
# adjust subplot within figure
fig.subplots_adjust(left=0.02,right=0.98,bottom=0.05,top=0.98)
plt.show()